In [1]:
# import python libraries
import numpy as np
import pandas as pd
import math
import datetime
import time
import copy
import gc
import sys
import csv
import os
from glob import glob
import matplotlib.pyplot as plt
import matplotlib
import SimpleITK as sitk
from multiprocessing import Process, Manager

from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

import tensorflow as tf
from tensorflow import keras
from keras.layers import AveragePooling3D
from tensorflow.keras.utils import to_categorical

from data_generate import *  # generate data functions
from KEM_SIMU import *       # KEM model
from simulation_auxiliary import *
sys.path.append('../')
sys.path.append('../Models/')
from utils import *
from GMM import *
from Kmeans import *

%matplotlib inline

In [2]:
# set the GPU
num_gpu = 1
gpu_name = 0
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_name}"

In [3]:
K = 3  # the number of classes

# the path of the CT data and the lung mask
lung_image_path = "/database/datasets/Classics/LUNA2016/IMAGES/"  # folder path for CT data
lung_mask_path = "/database/datasets/Classics/LUNA16-Mask/"       # folder path for lung mask

# all the absolute paths of the CT .mhd files
lung_image_file_list = glob(lung_image_path + "*.mhd")

# a dictionary containing the path information
stats_dict = {'lung_image_path': lung_image_path,
              'lung_mask_path': lung_mask_path,
              'lung_image_file_list': lung_image_file_list,
              }

# transform the dict so that it can be used in the following Process
manager = Manager()
stats_dict = manager.dict(stats_dict)

# Part 1. Bandwidth Selection

- Specify the `kernel sizes` (or `filter sizes`) and `bandwidth candidates` for the CV and REG methods.

In [4]:
kernel_size_list = np.arange(3, 12, 2)
coef_list = [0.3, 0.4, 0.5, 0.6, 0.7]

In [5]:
# CV: kernel size list and bandwidth candidate list
CV_kernel_size_list = []
CV_bandwidth_list = []
CV_coef_list = []
for kernel_size in kernel_size_list:
    for coef in coef_list:
        CV_kernel_size_list.append(kernel_size)
        CV_bandwidth_list.append(kernel_size / 512 * coef)
        CV_coef_list.append(coef)

# REG: kernel size list and bandwidth candidate list
REG_kernel_size_list = []
REG_bandwidth_list = []
for kernel_size in kernel_size_list:
    REG_kernel_size_list.append(kernel_size)
    REG_bandwidth_list.append(kernel_size / 512)
    
print(f"length of CV candidates: {len(CV_kernel_size_list)}")
print(f"length of REG candidates: {len(REG_kernel_size_list)}")

length of CV candidates: 25
length of REG candidates: 5


In [6]:
# save to stats_dict
stats_dict['CV_kernel_size_list'] = CV_kernel_size_list
stats_dict['CV_bandwidth_list'] = CV_bandwidth_list
stats_dict['CV_coef_list'] = CV_coef_list
stats_dict['REG_kernel_size_list'] = REG_kernel_size_list
stats_dict['REG_bandwidth_list'] = REG_bandwidth_list

- create csv file to record bandwidth selection results

In [7]:
# the path to record the Ch information
# Ch_path = f"./【results】/Ch-{datetime.date.today().isoformat()}.csv"
Ch_path = f"./【results】/Ch-2023-10-31.csv"

with open(Ch_path, 'w', newline='', encoding='utf-8') as f:
    csv_write = csv.writer(f)  
    csv_write.writerow([f"patient", "data_gen_time", 
                        "Ch_CV", "i_CV", "kernel_size_CV", "coef_CV", "time_CV", 
                        "Ch_REG", "time_REG", "model.coef0", "model.coef1"])

- randomly select 100 CT data to run the bandwidth selection experiments

In [8]:
# randomly select 100 patients for bandwidth selection
np.random.seed(0)
index_list = np.arange(0, len(lung_image_file_list), 1)
np.random.shuffle(index_list)
index_list = index_list[:100]

In [9]:
# if you have more than 1 GPU, then you can split the tasks equally into different GPUs
# if you have only 1 GPU, then all the tasks will be sent to this sole GPU
idx_increase = int(len(index_list) / num_gpu)
i = gpu_name
idx_start = i * idx_increase
idx_end = (i + 1) * idx_increase

if i == num_gpu - 1:
    idx_end = len(index_list)
print(f"GPU{gpu_name} with index list: [{idx_start}:{idx_end}]")
gpu_index_list = index_list[idx_start:idx_end]

GPU0 with index list: [0:100]


- bandwidth selection experiments
> The reason to use `Process` to run functions here:
> 
> If we run the `for` iterations on GPU without using `Process`, the GPU memory will be gradually exhausted and then raise memomry error. Thus, the iterations will be aborted. Consequently, I use `Process` to call the GPU computation. This practice will free the GPU memory for every iteration. I hope I make this point clear (in case I forget).

In [ ]:
for index in index_list:
    # set random seed
    np.random.seed(index)
    tf.random.set_seed(index)
    # add the information into the dict passing to processes
    stats_dict['index'] = index
    # use process to run the following function each iteration
    process_eval = Process(target=compute_Ch_4_cv_reg, 
                           args=(stats_dict, Ch_path, 0.8))
    process_eval.start()
    process_eval.join()

---------------WE ARE LOADING 150th PATIENT's CT with shape (292, 512, 512)---------------


2024-03-28 04:13:18.845468: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 04:13:19.414940: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:34:00.0, compute capability: 8.0
2024-03-28 04:13:20.864347: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


From function(generate_mu_sigma_new_version)
	 mu_const: Class bone[0.7755]; Class lung[0.4589]
##########[150]########## The 0th bandwidth from REG ####################
From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.0125 positions for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 2.002 seconds
	centers: [1.0218678 0.7761924 0.3713518]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 292, 512, 512, 1)
	pi_estimate: (3, 292, 512, 512, 1)
	mu_estimate: (3, 292, 512, 512, 1)
	sigma_estimate: (3, 292, 512, 512, 1)
From function(__init__): Initialize kernel successfully.
	kernel: (3, 3, 3, 1, 1)
From function(kem_algorithm): Receive max_steps: 20.
########################## STEP 0 ##########################
	 Current pik difference: 0.214011
From function(kem_algorithm): E step success.
+++ From m_step: add smooth_parameter to pi_estimate
From m_step: add smooth_parameter to sigma_est

2024-03-28 04:40:27.473233: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 04:40:27.993692: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:34:00.0, compute capability: 8.0
2024-03-28 04:40:29.094988: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


From function(generate_mu_sigma_new_version)
	 mu_const: Class bone[0.5230]; Class lung[0.2396]
##########[406]########## The 0th bandwidth from REG ####################
From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.01251 positions for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 1.03 seconds
	centers: [0.9979156  0.51148146 0.11891226]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 147, 512, 512, 1)
	pi_estimate: (3, 147, 512, 512, 1)
	mu_estimate: (3, 147, 512, 512, 1)
	sigma_estimate: (3, 147, 512, 512, 1)
From function(__init__): Initialize kernel successfully.
	kernel: (3, 3, 3, 1, 1)
From function(kem_algorithm): Receive max_steps: 20.
########################## STEP 0 ##########################
+++ From m_step: add smooth_parameter to pik_estimate
	 Current pik difference: 0.220635
From function(kem_algorithm): E step success.
	 Current pi difference: 0.0394189
	 Curre

...

From function(generate_mu_sigma_new_version)
	 mu_const: Class bone[0.6094]; Class lung[0.3632]
##########[701]########## The 0th bandwidth from REG ####################
From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.01251 positions for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 2.609 seconds
	centers: [1.0025358  0.60714203 0.20495851]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 375, 512, 512, 1)
	pi_estimate: (3, 375, 512, 512, 1)
	mu_estimate: (3, 375, 512, 512, 1)
	sigma_estimate: (3, 375, 512, 512, 1)
From function(__init__): Initialize kernel successfully.
	kernel: (3, 3, 3, 1, 1)
From function(kem_algorithm): Receive max_steps: 20.
########################## STEP 0 ##########################
+++ From m_step: add smooth_parameter to pik_estimate
	 Current pik difference: 0.219577
From function(kem_algorithm): E step success.
	 Current pi difference: 0.038361
	 Curre

Process Process-7:
Traceback (most recent call last):
  File "/root/miniconda3/envs/myconda/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/myconda/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/KEM/KEM_simulation/simulation_auxiliary.py", line 118, in compute_Ch_4_cv_reg
    kem_cv.kem_algorithm(max_steps=20, epsilon=1e-4, smooth_parameter=1e-20)
  File "/mnt/KEM/KEM_simulation/KEM_SIMU.py", line 95, in kem_algorithm
    difference = self.m_step(smooth_parameter)
  File "/mnt/KEM/KEM_simulation/KEM_SIMU.py", line 142, in m_step
    mu_estimate = tf.nn.conv3d(self.pik_estimate * self.training_data * self.position_mask, self.kernel,
  File "/root/miniconda3/envs/myconda/lib/python3.9/site-packages/tensorflow/python/util/traceback_utils.py", line 153, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/root/miniconda3/envs/myconda

---------------WE ARE LOADING 815th PATIENT's CT with shape (240, 512, 512)---------------


2024-03-28 05:17:35.316932: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 05:17:35.896314: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:34:00.0, compute capability: 8.0
2024-03-28 05:17:37.209023: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


From function(generate_mu_sigma_new_version)
	 mu_const: Class bone[0.7182]; Class lung[0.4320]
##########[815]########## The 0th bandwidth from REG ####################
From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.01249 positions for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 1.661 seconds
	centers: [1.0148996  0.7257482  0.31457683]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 240, 512, 512, 1)
	pi_estimate: (3, 240, 512, 512, 1)
	mu_estimate: (3, 240, 512, 512, 1)
	sigma_estimate: (3, 240, 512, 512, 1)
From function(__init__): Initialize kernel successfully.
	kernel: (3, 3, 3, 1, 1)
From function(kem_algorithm): Receive max_steps: 20.
########################## STEP 0 ##########################
	 Current pik difference: 0.216709
From function(kem_algorithm): E step success.
+++ From m_step: add smooth_parameter to pi_estimate
From m_step: add smooth_parameter to sigma

# Part 2. Comparison on the above 100 CT data

【Metrics】
- RMSE (with training position ratio=1)
    - $\pi$
    - $\mu$
    - $\sigma$
- Test ACC (with training position ratio=0.8)

【Methods】

- KEM (under personal optimal bandwidth):
    - CV
    - REG
- Kmeans
- GMM

## 2.1 Comparison: KEM

- create csv file to record KEM's results

In [10]:
# to_csv_path = f"./【results】/compare_metrics_KEM-{datetime.date.today().isoformat()}.csv"
to_csv_path = f"./【results】/compare_metrics_KEM-2023-11-01.csv"

with open(to_csv_path, 'w', newline = '', encoding='utf-8') as f:
    csv_write = csv.writer(f)  
    csv_write.writerow(["patient", "CV_ACC", "REG_ACC", 
                        "patient_again", "CV_pi_rmse", "CV_mu_rmse", "CV_sigma_rmse", 
                        "REG_pi_rmse", "REG_mu_rmse", "REG_sigma_rmse"])

- read in the bandwidth selection results in Part 1

In [11]:
# read the information of the randomly selected CT
random_info = pd.read_csv("./【results】/random100_info.csv")
# the bandwidth selection results from the previous python section
Ch_df = pd.read_csv(Ch_path)
Ch_df = Ch_df.drop_duplicates(subset=["patient"], keep="first")
# merge results
random_merge = pd.merge(random_info, Ch_df, how="left", on="patient")
random_merge.head()

,patient,depth,data_gen_time,Ch_CV,i_CV,kernel_size_CV,coef_CV,time_CV,Ch_REG,time_REG,model.coef0,model.coef1
0,patient_150,292,14.683044,0.111331,21.0,11.0,0.4,1456.803695,0.233042,401.203525,0.111005,0.000001
1,patient_406,147,14.988572,0.151400,23.0,11.0,0.6,613.941478,0.214797,149.646546,0.332126,0.000002
2,patient_513,110,11.642171,0.145258,23.0,11.0,0.6,760.700288,0.198921,182.091885,0.376583,0.000002
3,patient_101,197,20.568400,0.131556,22.0,11.0,0.5,784.775665,0.226474,197.654760,0.238026,0.000002
4,patient_584,383,25.538676,0.115730,21.0,11.0,0.4,860.322708,0.249160,300.563646,0.206875,0.000004


- If you wish to run this experiment using multiple GPUs or continue to run the experiment, you may consider to read in the results at hand.

In [12]:
exist_csv = pd.read_csv(to_csv_path)
exist_list = exist_csv.patient.apply(lambda x: int(x.split("_")[1]))
exist_list = list(exist_list)
Ch_list = list(Ch_df.patient.apply(lambda x: int(x.split("_")[1])))

len(exist_list), len(Ch_list)

(100, 100)

- comparison: KEM

In [25]:
for index in Ch_list:
    if index in exist_list:
        continue
    
    # set random seed
    np.random.seed(index)
    tf.random.set_seed(index)
    # extract the information of a selected patient
    data_index = random_merge.query(f'patient == "patient_{index}"')
    # add the information into the dict passing to processes
    stats_dict['index'] = index
    stats_dict["Ch_CV"] = float(data_index["Ch_CV"])
    stats_dict["Ch_REG"] = float(data_index["Ch_REG"])
    # use process to run the following function each iteration
    process_eval = Process(target=compute_metrics_4_KEM, args=(stats_dict, to_csv_path))
    process_eval.start()
    process_eval.join()

---------------WE ARE LOADING 406th PATIENT's CT with shape (147, 512, 512)---------------


2024-03-28 07:49:15.624068: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 07:49:16.193773: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:34:00.0, compute capability: 8.0
2024-03-28 07:49:17.305808: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


From function(generate_mu_sigma_new_version)
	 mu_const: Class bone[0.5230]; Class lung[0.2396]
#################### [Patient_406] ratio=0.8 ####################
#################### [406] CV ####################
From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.01251 data for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 0.9794 seconds
	centers: [0.9979156  0.51148146 0.11891226]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 147, 512, 512, 1)
	pi_estimate: (3, 1, 1, 1, 1)
	mu_estimate: (3, 147, 512, 512, 1)
	sigma_estimate: (3, 1, 1, 1, 1)
From function(__init__): Initialize kernel successfully.
	kernel: (5, 5, 5, 1, 1)
From function(kem_algorithm): Receive max_steps: 20.
########################## STEP 0 ##########################
+++ From m_step: add smooth_parameter to pik_estimate
	 Current pik difference: 0.220635
From function(kem_algorithm): E step success.
	 Current pi d

2024-03-28 07:50:30.484619: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 07:50:30.988387: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:34:00.0, compute capability: 8.0
2024-03-28 07:50:32.012345: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


From function(generate_mu_sigma_new_version)
	 mu_const: Class bone[0.7019]; Class lung[0.4545]
#################### [Patient_513] ratio=0.8 ####################
#################### [513] CV ####################
From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.01251 data for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 0.8335 seconds
	centers: [1.0108223  0.70681626 0.29644173]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 110, 512, 512, 1)
	pi_estimate: (3, 1, 1, 1, 1)
	mu_estimate: (3, 110, 512, 512, 1)
	sigma_estimate: (3, 1, 1, 1, 1)
From function(__init__): Initialize kernel successfully.
	kernel: (5, 5, 5, 1, 1)
From function(kem_algorithm): Receive max_steps: 20.
########################## STEP 0 ##########################
	 Current pik difference: 0.217572
From function(kem_algorithm): E step success.
	 Current pi difference: 0.0321163
	 Current mu difference: 0.003883

...

From function(generate_mu_sigma_new_version)
	 mu_const: Class bone[0.5347]; Class lung[0.2357]
#################### [Patient_145] ratio=0.8 ####################
#################### [145] CV ####################
From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.01251 data for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 0.9366 seconds
	centers: [0.9978675  0.5227723  0.12987456]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 147, 512, 512, 1)
	pi_estimate: (3, 1, 1, 1, 1)
	mu_estimate: (3, 147, 512, 512, 1)
	sigma_estimate: (3, 1, 1, 1, 1)
From function(__init__): Initialize kernel successfully.
	kernel: (5, 5, 5, 1, 1)
From function(kem_algorithm): Receive max_steps: 20.
########################## STEP 0 ##########################
+++ From m_step: add smooth_parameter to pik_estimate
	 Current pik difference: 0.220467
From function(kem_algorithm): E step success.
	 Current pi d

2024-03-28 08:05:33.991097: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 08:05:34.494691: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:34:00.0, compute capability: 8.0
2024-03-28 08:05:35.544890: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


From function(generate_mu_sigma_new_version)
	 mu_const: Class bone[0.7194]; Class lung[0.4398]
#################### [Patient_344] ratio=0.8 ####################
#################### [344] CV ####################
From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.01251 data for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 0.8316 seconds
	centers: [1.0146536  0.72410303 0.31549326]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 123, 512, 512, 1)
	pi_estimate: (3, 1, 1, 1, 1)
	mu_estimate: (3, 123, 512, 512, 1)
	sigma_estimate: (3, 1, 1, 1, 1)
From function(__init__): Initialize kernel successfully.
	kernel: (5, 5, 5, 1, 1)
From function(kem_algorithm): Receive max_steps: 20.
########################## STEP 0 ##########################
	 Current pik difference: 0.216508
From function(kem_algorithm): E step success.
	 Current pi difference: 0.0330734
	 Current mu difference: 0.003788

KeyboardInterrupt: 

Process Process-16:
Traceback (most recent call last):
  File "/root/miniconda3/envs/myconda/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/root/miniconda3/envs/myconda/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/KEM/KEM_simulation/simulation_auxiliary.py", line 213, in compute_metrics_4_KEM
    kem_reg.kem_algorithm(max_steps=20, epsilon=1e-4, smooth_parameter=1e-20)
  File "/mnt/KEM/KEM_simulation/KEM_SIMU.py", line 107, in kem_algorithm
    difference = self.m_step(smooth_parameter)
  File "/mnt/KEM/KEM_simulation/KEM_SIMU.py", line 147, in m_step
    if tf.reduce_sum(tf.cast(pi_estimate == 0, tf.float32)) > 0:  # pi_estimate cannot be 0 as the denominator in (2.8)
  File "/root/miniconda3/envs/myconda/lib/python3.9/site-packages/tensorflow/python/framework/ops.py", line 1134, in __bool__
    return bool(self._numpy())
  File "/root/miniconda3/envs/myconda/lib/pytho

## 2.1 Comparison: $k$-means and GMM

- create csv file to record $k$-means and GMM's results

In [13]:
to_csv_path = "./【results】/compare_metrics_Kmeans_GMM-2023-10-31.csv"

# with open(to_csv_path, 'w', newline = '', encoding='utf-8') as f:
    csv_write = csv.writer(f)  
    csv_write.writerow(["patient", "Kmeans_acc", "GMM_acc",
                        "patient_again", "Kmeans_pi_rmse","Kmeans_mu_rmse", "Kmeans_sigma_rmse",
                        "GMM_pi_rmse", "GMM_mu_rmse", "GMM_sigma_rmse"])

- If you wish to run this experiment using multiple GPUs or continue to run the experiment, you may consider to read in the results at hand.

In [14]:
exist_csv = pd.read_csv(to_csv_path)
exist_list = exist_csv.patient.apply(lambda x: int(x.split("_")[1]))
exist_list = list(exist_list)
len(exist_list)

0

- comparison: $k$-means and GMM

In [ ]:
failed_index_list = []  # if there are failed cases

for index in index_list:
    if index in exist_list:
        continue
    
    # set random seed
    np.random.seed(index)
    tf.random.set_seed(index)
    # add the information into the dict passing to processes
    stats_dict['index'] = index
    # use process to run the following function each iteration
    process_eval = Process(target=compute_metrics_4_GMM_Kmeans, 
                           args=(stats_dict, to_csv_path))
    process_eval.start()
    process_eval.join()
    if process_eval.exitcode == 1:
        print(f"\n with index:{index}")
        failed_index_list.append(index)
print()
print(failed_index_list)

---------------WE ARE LOADING 150th PATIENT's CT with shape (292, 512, 512)---------------


2024-03-28 08:50:11.763579: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 08:50:12.327448: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:34:00.0, compute capability: 8.0
2024-03-28 08:50:13.802004: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


From function(generate_mu_sigma_new_version)
	 mu_const: Class bone[0.7755]; Class lung[0.4589]
#################### [Patient_150] ratio=0.8 ####################
From function(__init__): Randomly pick 0.0125 data for kmeans.


/root/miniconda3/envs/myconda/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1305: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params(X)


From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.01251 data for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 1.575 seconds
	centers: [1.0209466  0.7743281  0.37109444]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 292, 512, 512, 1)
	pi_estimate: (3, 1, 1, 1, 1)
	mu_estimate: (3, 1, 1, 1, 1)
	sigma_estimate: (3, 1, 1, 1, 1)
From function(gmm_algorithm): Receive max_steps: 20.
########################## STEP 0 ##########################
	 Current pik difference: 0.214103
From function(gmm_algorithm): E step success.
pi_estimator: [0.4399774  0.26800278 0.29202855]

mu_estimate: [1.0209466  0.7743281  0.37109444]
sigma_estimator: [0.08238964 0.08205338 0.08708534]
	 Current pi difference: 0.00578238
	 Current mu difference: 2.20206e-06
	 Current sigma difference: 0.00115061
From function(gmm_algorithm): M step success.
From function(gmm_algorithm): difference: 0.00693519.
---This 

...

2024-03-28 08:54:27.464336: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 08:54:28.024647: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:34:00.0, compute capability: 8.0
2024-03-28 08:54:29.870397: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


From function(generate_mu_sigma_new_version)
	 mu_const: Class bone[0.7410]; Class lung[0.4436]
#################### [Patient_395] ratio=0.8 ####################
From function(__init__): Randomly pick 0.01251 data for kmeans.


/root/miniconda3/envs/myconda/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1305: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params(X)


From function(__init__): Initialize mu via kmeans(with K=3)
From function(__init__): Randomly pick 0.01251 data for kmeans.
From function(__init__): KMeans(with K=3) success, with time: 2.694 seconds
	centers: [1.0186088  0.74976146 0.33858833]
From function(__init__): Initialize parameters successfully.
	pik_estimate:(3, 461, 512, 512, 1)
	pi_estimate: (3, 1, 1, 1, 1)
	mu_estimate: (3, 1, 1, 1, 1)
	sigma_estimate: (3, 1, 1, 1, 1)
From function(gmm_algorithm): Receive max_steps: 20.
########################## STEP 0 ##########################
	 Current pik difference: 0.215468
From function(gmm_algorithm): E step success.
pi_estimator: [0.47869483 0.26312444 0.2581947 ]

mu_estimate: [1.0186088  0.74976146 0.33858833]
sigma_estimator: [0.0848569  0.08673685 0.08921658]
	 Current pi difference: 0.0105684
	 Current mu difference: 1.65249e-07
	 Current sigma difference: 0.00136751
From function(gmm_algorithm): M step success.
From function(gmm_algorithm): difference: 0.011936.
---This ite

/root/miniconda3/envs/myconda/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1305: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  super()._check_params(X)
